# Classification. Part 2
---
Author: Anatoliy Durkin

Updated: 17.03.2025

---
В данном ноутбуке будут рассмотрены несколько моделей классификации, новые функции для метрик и сами новые метрики, а также будет уделено внимание подбору гиперпараметров для моделей.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

В прошлом ноутбуке мы писали функцию, выводившую все необходимые нам метрики вместе. Мы импортировали несколько функций, но в этом ноутбуке используем только две: `confusion_matrix` выводит матрицу ошибок, а `classification_report` выдает остальные метрики, увидим уже на примере. Зачем нужен `fill_diagonal`, увидим в конце ноутбука.

In [ ]:
def metrics(target, pred, fill=False):
    print(classification_report(target, pred))
    matrix = np.array(confusion_matrix(target, pred))
    if fill:
        np.fill_diagonal(matrix, 0)
    sns.heatmap(matrix, annot=True, fmt='.0f')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Матрица ошибок')

## Naive Bayes
---

Знаете ли вы метод "съешь лягушку"? Это о том, что нужно выполнить самую неприятную задачу сразу, не откладывая. Так что съедим лягушку и слегка окунёмся в NLP, или Natural language processing - обработка естественного языка.

В этой сфере может быть множество задач классификации, например, отделение спама. Но мы рассмотрим задачу определения кликбейта среди новостных заголовков.

> Кликбейт (англ. clickbait от click «щелчок» + bait «приманка») — уничижительный термин, описывающий веб-контент, целью которого является получение дохода от онлайн-рекламы, особенно в ущерб качеству или точности информации.

А как модель используем наивный байессовский классиикатор.

Итак, познакомимся с данными.

In [ ]:
titles = pd.read_csv('titles_data.csv', sep=';')

In [ ]:
titles

Поделим выборку на обучающую и тестовую.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(titles['titles'], titles['target'], test_size=0.2, random_state=42)

Конечно, модели не могут просто так работать с обычным текстом. Необходимы какие-то преобразования. Все модели спокойно работают с числами, значит нужно превратить текст в числа. И тут есть два важных процесса: токенизация и векторизация.

> Токенизация — процесс разбиения текстового документа на отдельные слова, которые называются токенами.

> Векторизация – это термин, обозначающий классический подход к преобразованию входных данных из их исходного формата (например, текста) в векторы действительных чисел, которые понятны моделям машинного обучения.

---
Как мы применим это к нашим данным?

Например, для определения кликбейта можно посчитать, как часто слова встречаются в заголовках. Скорее всего определенные слова могут сигнализировать нам о кликбейте.

В этом нам может помочь `CountVectorizer`, который считает, сколько раз каждое слово встречается в определенной строке.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
txt = ['мама мыла раму',
      'мама чинила раму',
      'мама мыла, мама чинила']

In [ ]:
vectorizer = CountVectorizer()
txt = vectorizer.fit_transform(txt)

In [ ]:
# результат
txt.toarray()

In [ ]:
# словарь
vectorizer.get_feature_names_out()

Мы получаем таблицу, в которой каждый столбец соответствует слову из сформированного словаря, а строка - фразе из исходного корпуса (набора) текстов. Если взглянуть на таблицу и на словарь, видно, что на пересечении находится количество, сколько раз встречается определенное слово в определенном тексте.

Словарь составляется автоматически в лексикографическом порядке.

Уже здесь вы можете указать на значительное упущение, сделанное мной намеренно, и которое может создать сложности при дальнейшей работе. Об этом поговорим позже, а пока применим векторизатор к нашим даным.

In [ ]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

В библиотеке `sklearn` есть несколько моделей наивного байесса, мы возьмем полиномиальную, она хорошо работает как раз с данной векторизацией.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model = MultinomialNB()
model.fit(X_train_vec, y_train)

In [ ]:
y_pred = model.predict(X_test_vec)
metrics(y_test, y_pred)

Не самые плохие результаты.

Но можно ли улучшить показатели модели? И за счет чего?

---
`CountVectorizer` будет создавать выбросы для слишком асто встречающихся слов, из-за чего будут занижаться значения для некоторых важных слов. Чтобы этого избежать, попробуем использовать другой векторизатор, TF-IDF.

> Термины "TF" (Term Frequency) и "IDF" (Inverse Document Frequency).
>
> TF (Частота термина) обозначает, насколько часто определенное слово появляется в данном документе. Таким образом, TF измеряет важность слова в контексте отдельного документа.
>
> IDF (Обратная частота документа) измеряет, насколько уникально слово является по всей коллекции документов. Слова, которые появляются в большинстве документов, имеют низкое IDF, так как они не вносят большой информационной ценности.

Формула TF-IDF комбинирует понятия TF и IDF, чтобы вычислить важность каждого слова в каждом документе. Формально, формула выглядит следующим образом:

> **TF-IDF(t, d) = TF(t, d) * IDF(t)**

Где:

TF(t, d) - Частота термина (TF) для слова "t" в документе "d".

IDF(t) - Обратная частота документа (IDF) для слова "t".

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Как выглядит таблица и словарь? Напишите код, чтобы посмотреть.

In [ ]:
# Корпус текстов можете заменить по желанию
txt = ['мама мыла раму',
      'мама чинила раму',
      'мама мыла, мама чинила'] 

In [ ]:
# Ваш код
...

А теперь можно обучить модель с использованием данного векторизатора.

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)
metrics(y_test, y_pred)

Изменения незначительны. На самом деле это связано с устройством самой модели. Даже в документации вы можете прочитать, что модель работает с дискретными целыми значениями. А TF-IDF, как мы видели, дробный. Однако, отметим, что всё равно модель успешно справилась.

---
Что ещё мы можем сделать?

Конечно, мы пропустили первый и важный этап обработки текста. Токенизация. Нам нужно превратить сплошной текст в набор токенов. Да, если разбитьь текст по пробелу, это тоже токены, однако, у нас в текстах могут встречаться числа и символы (иногда они нужны, но зачастую нет), а также одно и то же слово может быть представлено в различных словоформах (приставки, суффиксы, окончания). Со всем этим нужно работать.

Для борьбы со словоформами можно использовать два разных алгоритма:

> Лемматиза́ция — процесс приведения словоформы к лемме — её нормальной (словарной) форме.

> Сте́мминг (англ. stemming — находить происхождение) — это процесс нахождения основы слова для заданного исходного слова.

Мы используем лемматизацию, для этого подгрузим библиотеку `pymystem3`, разработанную в Яндекс.

In [ ]:
from pymystem3 import Mystem
import re
import nltk
from nltk.corpus import stopwords

Мы создадим лемматизатор, а также загрузим список стоп-слов. Это такие слова, которые обычно не несут никакой смысловой нагрузки (союзы, предлоги, etc.).

In [ ]:
m = Mystem()
nltk.download('stopwords')
stop_words = list(stopwords.words('russian'))

Вот что получается при работе лемматизатора:

In [ ]:
m.lemmatize('Интересного текста много не бывает')

Также накинем регулярные выражения, чтоы оставить в каждом тексте только русские буквы. И лемматизируем полученные тексты.

In [ ]:
train_ru = [" ".join(re.sub(r'[^а-яА-ЯёЁ]', ' ', text).split()) for text in X_train.values.astype('U')]
train_full = ' br '.join(train_ru)
train_lem = (''.join([word for word in m.lemmatize(train_full) if word != '\n'])).split(' br ')

test_ru = [" ".join(re.sub(r'[^а-яА-ЯёЁ]', ' ', text).split()) for text in X_test.values.astype('U')]
test_full = ' br '.join(test_ru)
test_lem = (''.join([word for word in m.lemmatize(test_full) if word != '\n'])).split(' br ')

In [ ]:
train_lem

А теперь к уже обработанному корпусу применим векторизатор и построим модель.

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
X_train_lem = tfidf_vectorizer.fit_transform(train_lem)
X_test_lem = tfidf_vectorizer.transform(test_lem)

In [ ]:
model = MultinomialNB()
model.fit(X_train_lem, y_train)
y_pred = model.predict(X_test_lem)
metrics(y_test, y_pred)

Результат стал даже чуть хуже, но вспомним об особенностях работы модели и попробуем это исправить. Попробуйте получить лучшие метрики.

In [ ]:
# Ваш код
...

## Logistic Regression

---
Переходим к следующей модели.

Для обучения возьмем данные по диабету.

In [ ]:
diabetes = pd.read_csv('diabetes_dataset.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop(['Outcome'], axis=1), diabetes['Outcome'], test_size=0.2, random_state=42)

Для логистической регрессии обязательно нужно нормализовать данные.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression(random_state=42).fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
metrics(y_test, y_pred)

Однако, вы помните, что классы в этом датасете несббалансированны, а это не очень хорошо. Но логистическая модель может сама обучаться с поправкой на дисбаланс, главное указать это в аргументах.

In [ ]:
clf = LogisticRegression(class_weight='balanced', random_state=42).fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
metrics(y_test, y_pred)

Рассмотрим ещё две метрики, позволяющие оценить качество модели. ROC-кривая и ROC-AUC.

---
Для построения этих показателей необходимо рассмотреть изменение порога классификации. Поскольку у нас всего два класса, можно рассматривать вероятность получения класса "1". По умолчанию порог равен 0.5 - получили значение выше 0.5, класс "1", ниже - "0".

Но этот порог можно изменять. Как будут вести себя точность и полнота, если снизить порог до 0.3? А если повысить до 0.8?

У логистической регрессии есть метод `predict_proba`, который автоматически считает вероятности для классов при различном пороге классификации.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
proba = clf.predict_proba(X_test)
proba_one = proba[:, 1]

In [ ]:
proba

In [ ]:
proba_one

Нам нужны два важных значения. Вспомним матрицу ошибок

---
|           | Predicted: 0 | Predicted:1 |
|-----------|:------------:|:-----------:|
| Actual: 0 | TN           | FP          |
| Actual: 1 | FN           | TP          |

---
$TPR = \frac{TP}{TP+FN}$

$FPR = \frac{FP}{FP+TN}$

Что отражают TPR и FPR?

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, proba_one)

In [ ]:
plt.figure()
plt.plot(fpr, tpr)

# ROC-кривая случайной модели (выглядит как прямая)
plt.plot([0, 1], [0, 1], linestyle='--')

plt.xlim([0, 1])
plt.ylim([0, 1])

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')

Для данной модели ROC-кривая почти идеальна, это замечательно. А метрика AUC-ROC - это площадь под ROC-кривой.

In [ ]:
auc_roc = roc_auc_score(y_test, y_pred)
print('AUC-ROC:',auc_roc)

## Random Forest

---
Переходим к ансамблевому методу обучения.

Что такое ансамбль? Нет, не песни и пляски, а применительно к нашей теме.

> Ансамблевый метод - метод машинного обучения, где несколько моделей обучаются для решения одной и той же проблемы и объединяются для получения лучших результатов.

---
Есть два наиболее популярных класса ансаблевых методов: бэггинг и бустинг. О первом мы поговорим сейчас, а второй оставим на будущие темы.

> Бэггинг (от англ. bootstrap aggregating, бутстрэп-агрегирование) — ансамблевый метаалгоритм, предназначенный для улучшения стабильности и точности алгоритмов машинного обучения, используемых в задачах классификации и регрессии. Алгоритм также уменьшает дисперсию и помогает избежать переобучения.

Если задан стандартный тренировочный набор $D$ размера $n$, бэггинг образует $m$ новых тренировочных наборов $D_i$, каждый размером $n′$, путём выборки из $D$ равномерно и с возвратом. При семплинге с возвратом некоторые наблюдения могут быть повторены в каждой $D_i$. Этот вид семплинга известен как бутстрэп-семплинг. Эти $m$ моделей сглаживаются с помощью вышеупомянутых $m$ бутстрэп-выборок и комбинируются путём усреднения (для регрессии) или голосования (для классификации).

---
Именно так и устроен случайный лес. Но прежде чем обратиться к обучению моделей, пара слов о подготовке данных и выборе признаков. Используем всё те же данные по диабету.

In [ ]:
diabetes

Конечно, мы схитрим и изначально не будем использовать признак `FamilyHistory`, чтобы заведемо снизить качество модели.

Но у нас остаётся ещё 15 признаков. Да, это не так уж много, это не 200 признаков, когда их число точно хочется сократить. Но иногда и при небольшом количестве признаков стоит подумать и, возможно, от чего-то избавиться.

Один из простых шагов - убрать сильно коррелированные переменные.

Сильно коррелированные друг с другом переменные дают модели одну и ту же информацию, следовательно, для анализа не нужно использовать их все. Например, если набор данных (dataset) содержит признаки «Время в сети» и «Использованный трафик», можно предположить, что они будут в некоторой степени коррелированы, и мы увидим сильную корреляцию, даже если выберем непредвзятый образец данных. В таком случае в модели нужна только одна из этих переменных. Если использовать обе, то модель окажется переобучена (overfit) и предвзята относительно одного отдельного признака.

Посмотрите на корреляционную матрицу и поищите, какие признаки могут быть удалены.

In [ ]:
plt.figure(figsize=(12,10))
plt.title('Корреляционная матрица')
sns.heatmap(diabetes.corr(), cmap='bwr', center=0, annot=True, fmt='.2f')

Обучим простую модель случайного леса. Исключаю `FamilyHistory` для снижения метрик, глубину деревьев беру рандомно, число никак не подбиралось.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop(['Outcome', 'FamilyHistory'], axis=1), diabetes['Outcome'], test_size=0.2, random_state=42)

In [ ]:
rf = RandomForestClassifier(max_depth=8, random_state=42).fit(X_train, y_train)

In [ ]:
metrics(y_test, rf.predict(X_test))

А теперь повторим процесс, но из признаков исключим также `HbA1c`. Он сильно коррелирует с `Glucose`, а значит, они несут очень схожую информацию для модели. Посмотрим, к чему это приведет.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop(['Outcome', 'FamilyHistory', 'HbA1c'], axis=1), diabetes['Outcome'], test_size=0.2, random_state=42)

In [ ]:
rf = RandomForestClassifier(max_depth=8, random_state=42).fit(X_train, y_train)

In [ ]:
metrics(y_test, rf.predict(X_test))

Изменения почти отсутсвуют, но всё же есть, и в лучшую сторону. Значит такой подход к выбору признаков работает.

Есть множество способов отбора признаков, о некоторых поговорим, когда будем работать с регрессией.

А ещё можно посмотреть на важность признаков в обученной модели с помощью метода `feature_importances_`.

In [ ]:
pd.Series(data=rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

Попробуйте построить ту же модель, исключив неважные признаки, изменится ли качество.

In [ ]:
# Ваш код
...

Также посмотрим на ROC-кривую для такой модели (у случайного леса тоже есть такие методы).

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, rf.predict_proba(X_test)[:, 1])

In [ ]:
plt.figure()
plt.plot(fpr, tpr)

# ROC-кривая случайной модели (выглядит как прямая)
plt.plot([0, 1], [0, 1], linestyle='--')

plt.xlim([0, 1])
plt.ylim([0, 1])

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')

In [ ]:
auc_roc = roc_auc_score(y_test, rf.predict(X_test))
print(auc_roc)

## GridSearchCV

---
В прошлый раз мы говорили о кросс-валидации, которая помогает получить более правильные метрики при усреднении нескольких моделей.

Но куда важнее для модели подобрать нужные гиперпараметры, от этого зависит очень многое, метрики могут отличаться значительно. Самое простое, что можно сделать - ручной перебор, меняем гиперпараметр, обучаем, проверяем, и так по кругу, пока не получим оптимальный результат. Но, это скучно и утомительно, поэтому можно воспользоваться `GridSearchCV`, который проделает тот же процесс самостоятельно. Более того, он попутно проводит и кросс-валидацию.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop(['Outcome', 'FamilyHistory'], axis=1), diabetes['Outcome'], test_size=0.2, random_state=42)

In [ ]:
rf = RandomForestClassifier()

In [ ]:
from sklearn.model_selection import GridSearchCV

В `SridSearchCV` мы передаем нужную нам модель и перечень гиперпараметров, которые хотим варьировать, с желаемыми значениями. Также можно указать, на сколько частей бить данные при кросс-валидации, и желаемую метрику. Сколько раз будет обучаться модель при следующих параметрах?

In [ ]:
parameters = {'max_depth': [5, 8], 'n_estimators': [50, 100], 'random_state': [42]}
clf = GridSearchCV(rf, parameters, cv=5, scoring='roc_auc')
clf.fit(X_train, y_train)

А после обучения можно посмотреть на лучшую оценку, лучшую модель и просто посмотреть на все результаты обучения.

In [ ]:
clf.best_score_

In [ ]:
clf.best_params_

In [ ]:
clf.cv_results_

Попробуйте использовать `GridSearchCV` с тем набором признаков, который считаете оптимальным. Подберите лучший набор гиперпараметров, выберите метрику и посмотрите, какого результата сможете достичь.

Метрики можно посмотреть тут: [scoring-parameter](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

In [ ]:
%time
# этот код позволяет вывести итоговое время выполнения ячейки, если вам интересно засечь время обучения

In [ ]:
# Ваш код
...

## Multiclass classification

---
Всюду до этого мы рассматривали случаи бинарной классификации, где у нас были только два класса. Однако зачастую нам нужно классифицировать данные по нескольким группам. Есть два подхода к использванию бинарных классификаторов для многоклассовой классификации.

> Один против всех (One-versus-all, OvA или один против остальных, One-versus-rest, OvR). Для каждого класса строится один бинарный классификатор. При этом примеры класса определяются как «положительные», а всех других — как «отрицательные». Итоговый результат формируется по принципу «победитель получает все»: объект будет отнесен к классу, для которого бинарный классификатор даст большее число «положительных» примеров.

> Один против одного (One versus One, OvO). Строится $k(k−1)$ классификаторов, позволяющих различить любую пару примеров разных классов. Алгоритм просматривает все пары примеров с разными метками классов и для каждой решает бинарную задачу $f_{ij}$. В каждом случае для пар $(i,j)$ положительные — все примеры с метками $i$, а отрицательными — с $j$.

Какие недостатки можно отметить у обоих способов?

Но многие модели просто умеют работать с несколькими классами. Например, случайный лес делает это легко.

Для демонстрации загрузим новые данные.

Этот набор данных содержит исчерпывающую информацию о 2392 учениках старших классов, в которой подробно описываются их демографические данные, привычки в учебе, участие родителей, внеклассные мероприятия и успеваемость. Целевая переменная, `GradeClass`, классифицирует оценки учеников по отдельным категориям, предоставляя надежный набор данных для образовательных исследований, прогнозного моделирования и статистического анализа.

In [ ]:
stud = pd.read_csv('Student_performance_data.csv')

In [ ]:
stud

Теперь обучим модель и посмотрим на метрики.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(stud.drop(['GradeClass'], axis=1), stud['GradeClass'], test_size=0.2, random_state=42)

In [ ]:
rf = RandomForestClassifier(max_depth=8).fit(X_train, y_train)

In [ ]:
metrics(y_test, rf.predict(X_test))

Как видим, теперь для каждого класса у нас есть набор метрик, а также общие оценки.

И тепловая карта с матрицей ошибок выглядит лучше. Однако на ней заметен дисбаланс классов. А когда мы хотим оценить, где модель делает наибольшие ошибки, нам лучше занулить диагональ. Для этого в нашей функции `metrics` предусмотрен код.

In [ ]:
metrics(y_test, rf.predict(X_test), True)

Тут видно, где модель ошибается боьше всего.

В завершение предлагаю вам настроить и эту модель, подобрав лучшие гиперпараметры.

In [ ]:
# Ваш код
...